Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
print("Pipeline SDK-specific imports completed")

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure blob storage)
# def_blob_store = ws.get_default_datastore()
def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

In [ ]:
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Aml Compute attached")


In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
cd = CondaDependencies()
cd.add_pip_package("azureml-sdk")
cd.add_pip_package("azureml-widgets")
cd.add_pip_package("azureml-datadrift")
cd.add_pip_package("azureml-dataprep[pandas]")
cd.add_pip_package("pyarrow")
cd.add_conda_package("scikit-learn")
run_config.environment.python.conda_dependencies = cd


In [ ]:
keyvault = ws.get_default_keyvault()

datadrift_step = PythonScriptStep(
    name='Datadrift',
    script_name='datadrift.py',
    arguments=['--tenant_id', keyvault.get_secret('TENANT-ID'),
               '--app_id', keyvault.get_secret('APP-ID'),
               '--client_secret', keyvault.get_secret('CLIENT-SECRET')],
    compute_target=aml_compute,
    runconfig=run_config,
    source_directory=os.path.join(os.getcwd(), 'datadrift_script'),
    allow_reuse=False
)

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=[datadrift_step])
print ("Pipeline is built")

In [ ]:
pipeline_run1 = Experiment(ws, 'Data_dependency').submit(pipeline1)
print("Pipeline is submitted for execution")

In [ ]:
pipeline_run1.wait_for_completion(show_output=True)